# **1. REPROYECCIÓN MULTIPLES DATOS RASTER**

Permite automatizar tareas de preprocesamiento esenciales en el ámbito de la programación SIG, asegurando que todos los conjuntos de datos estén espacialmente alineados para análisis posteriores, como la construcción de modelos machine learning.

## 1.1 Importación de librerias y configuración de rutas

In [3]:
import os
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject
import numpy as np

# Ruta base en entorno Docker (ajustado para contenedor)
carpeta = 'C:/Users/Equipo/SemDocker/ProyectoProgramacionSIG/Data/Imagenes_25_07_12_v2'
dem_path = os.path.join(carpeta, 'DEM9377.tif')
log_path = os.path.join(carpeta, 'log_resampleo.txt')

 Procesado: gedi.tif (1 banda(s))
 Procesado: mapbiomas_amacayacu_2023.tif (1 banda(s))
 Procesado: SAR_PW.tif (2 banda(s))
 Procesado: sentinel2_index_proye.tif (7 banda(s))

 Todos los archivos fueron procesados sin errores.


## 1.2 Apertura del DEM de referencia y lista de archivos a procesar

In [ ]:
# Abrir DEM9377 para usar como plantilla
with rasterio.open(dem_path) as dem_ref:
    dst_crs = dem_ref.crs
    dst_transform = dem_ref.transform
    dst_width = dem_ref.width
    dst_height = dem_ref.height
    dst_profile = dem_ref.profile.copy()

# Lista de archivos .tif a procesar (excluyendo los ya procesados)
imagenes = [f for f in os.listdir(carpeta)
            if f.endswith('.tif') and f not in ['DEM.tif', 'DEM9377.tif']]

## 1.3 Bucle principal de procesamiento

In [ ]:
# Inicializar log de errores
errores = []

for nombre in imagenes:
    entrada_path = os.path.join(carpeta, nombre)
    salida_path = os.path.join(carpeta, nombre.replace('.tif', '_resampleado9377.tif'))

    # Selección del método de resampleo
    if nombre in ['mapbiomas_amacayacu_2023.tif', 'gedi.tif']:
        metodo_resampleo = Resampling.nearest
    else:
        metodo_resampleo = Resampling.bilinear

    try:
        with rasterio.open(entrada_path) as src:
            # Determinar el tipo de dato
            tipo = src.dtypes[0]
            nodata = src.nodata

            # Asignar nodata válido si no está definido
            if nodata is None:
                if tipo == 'uint8':
                    nodata = 255
                elif tipo == 'uint16':
                    nodata = 65535
                elif tipo in ['int16', 'int32']:
                    nodata = -9999
                elif tipo.startswith('float'):
                    nodata = -9999.0
                else:
                    raise ValueError(f"Tipo de dato no reconocido: {tipo}")

            # Crear array de salida lleno con nodata
            data_resampled = np.full(
                (src.count, dst_height, dst_width),
                nodata,
                dtype=tipo
            )

            # Reproyección banda por banda
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=data_resampled[i - 1],
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=metodo_resampleo,
                    src_nodata=nodata,
                    dst_nodata=nodata
                )

            # Actualizar metadatos
            new_profile = dst_profile.copy()
            new_profile.update({
                'height': dst_height,
                'width': dst_width,
                'transform': dst_transform,
                'count': src.count,
                'dtype': tipo,
                'nodata': nodata
            })

            # Guardar archivo reproyectado
            with rasterio.open(salida_path, 'w', **new_profile) as dst:
                dst.write(data_resampled)

            print(f' Procesado: {nombre} ({src.count} banda(s))')

    except Exception as e:
        errores.append(f'{nombre}: {str(e)}')
        print(f' Error con {nombre}: {str(e)}')

# Guardar log de errores
if errores:
    with open(log_path, 'w') as f:
        for error in errores:
            f.write(f'{error}\n')
    print(f'\n Log guardado en: {log_path}')
else:
    print('\n Todos los archivos fueron procesados sin errores.')